In [1]:
%run ../0_Data/0_DataLoader.ipynb

[nltk_data] Downloading package punkt to /home/celeste/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/celeste/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/celeste/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [73]:

import numpy as np

#Pipelines
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion

#Tuning de Parametros
from sklearn.model_selection import GridSearchCV

#Clasificador
from sklearn.linear_model import LogisticRegression # Classifier using Logistic Regression.
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

#Metrics
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import zero_one_loss
from sklearn.metrics import f1_score

In [2]:

class ModelEvaluator:
    def __init__(self, model, params, params_names ,data_x, data_y, clean_methods, metrics, metrics_names):
        self.model = model
        self.data_x = data_x
        self.data_y = data_y
        self.clean_methods = clean_methods
        self.metrics = metrics
        

    def evaluate ():
        
        for clean_method in clean_methods:
            if (clean_method != None):
                data_x_clean = clean_method(data_x)
            else:
                data_x_clean = data_x
            model.train(data_x_clean)
            y_pred = model.predict(data_x_clean)
            
            
        
    
    

In [116]:
def evaluate (model, data_x, data_y, clean_methods, metrics):
    
    clean_names = []
    for clean_method, cm_name in clean_methods:
        if (clean_method != None):
            clean_names.append(cm_name)
        else:
            clean_names.append('Ninguno')
    metric_names = [m[1] for m in metrics]
    columns = ['Modelo', 'Metodo de limpieza', 'Metrica', 'Valor'] 
    results = pd.DataFrame (columns = columns ) 
    
    for clean_method, cm_name in clean_methods:
            if (clean_method != None):
                #print ('\tMetodo de limpieza: ' + cm_name)
                data_x_clean = data_x.apply(lambda x: clean_method(x))
            else:
                #print ('\tMetodo de limpieza: Ninguno')
                data_x_clean = data_x
            model.train(data_x_clean, data_y)
            y_pred = model.predict(data_x_clean)
            
            for metric, mname in metrics:
                m = metric(data_y, y_pred)
                #print ('\t\tMetrica ',mname, ': ',m)
                df = pd.DataFrame([[model.getName(),cm_name,mname,m]], columns = columns)
                results = results.append(df, ignore_index = True)
                #print (df)
    return results
                

In [115]:

columns = ['a', 'b', 'c']

results = pd.DataFrame (columns = columns ) 
df = pd.DataFrame([[1,2,3]], columns = columns)
results = results.append(df, ignore_index = True)
results = results.append(df, ignore_index=True)
results

,a,b,c
0,1,2,3
1,1,2,3


In [62]:
grid_parametros = {'clf__penalty':['l1', 'l2'] ,
                   'clf__C': np.arange(0.06,0.08,0.01),
                  'vectorizador__ngram_range': [(1,1),(1,2)]}

lrm = LogisticRegressionModel(grid_parametros, '')

In [79]:
clean_methods = [(None, 'Ninguno'),(clean_text_sentiment_analysis, 'clean_text_sentiment_analysis'), (limpiar_texto, 'limpiar_texto'), (clean_text_simple,'clean_text_simple')]
metrics = [(accuracy_score, 'Accuracy'),
          (zero_one_loss, 'Loss'),
          (f1_score, 'F1')]

In [60]:
class LogisticRegressionModel:
    model = None
    grid_params = None
    params_description = None
    
    def __init__(self, grid_params , params_description):
        self.grid_params = grid_params
        self.params_description = params_description
        
    def getName(self):
        return 'Logistic Regression ' + self.params_description
    
    def train(self, x, y):
        pipeline_ngrams_LR = Pipeline([('vectorizador', CountVectorizer()),('clf', LogisticRegression())])
        self.model = GridSearchCV(pipeline_ngrams_LR, self.grid_params,cv=5, n_jobs=-1)
        self.model.fit(x, y)
        
        
    def predict(self, x):
        return self.model.predict(x)






In [23]:
x_train_original, train_y = get_train_x_y_original()

x_test_original = get_test_x_original()

In [117]:
res = evaluate(lrm, x_train_original, train_y, clean_methods, metrics )
res

,Modelo,Metodo de limpieza,Metrica,Valor
0,Logistic Regression,Ninguno,Accuracy,0.860633
1,Logistic Regression,Ninguno,Loss,0.139367
2,Logistic Regression,Ninguno,F1,0.821771
3,Logistic Regression,clean_text_sentiment_analysis,Accuracy,0.899120
4,Logistic Regression,clean_text_sentiment_analysis,Loss,0.100880
5,Logistic Regression,clean_text_sentiment_analysis,F1,0.872383
6,Logistic Regression,limpiar_texto,Accuracy,0.858663
7,Logistic Regression,limpiar_texto,Loss,0.141337
8,Logistic Regression,limpiar_texto,F1,0.816257
9,Logistic Regression,clean_text_simple,Accuracy,0.914357
